<a href="https://colab.research.google.com/github/yxwang0812/TurnerLab/blob/main/colabs/hs488_snvs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch variant scoring

In this notebook, we demonstrate how to score many variants using AlphaGenome.

To prepare numerous variants for scoring, provide the following information as columns in a tab-separated Variant Call Format (VCF) file:
- `variant_id`: A unique identifier for each variant.
- `CHROM`: Chromosome, specified as a string beginning with 'chr' (e.g., 'chr1').
- `POS`: Integer representing the base pair position (1-based; build hg38 (human) or mm10 (mouse) - see [FAQ](https://www.alphagenomedocs.com/faqs.html#how-do-i-define-a-variant)).
- `REF`: The reference nucleotide sequence at the specified position.
- `ALT`: The alternate nucleotide sequence at the specified position.



```{tip}
Open this tutorial in Google colab for interactive viewing.
```

In [1]:
from IPython.display import clear_output
! pip install alphagenome
clear_output()

In [12]:
from io import StringIO
from alphagenome.data import genome
from alphagenome.models import dna_client, variant_scorers
from alphagenome.visualization import plot_components
from google.colab import data_table, files
from google.colab import userdata
import pandas as pd
from tqdm import tqdm

data_table.enable_dataframe_formatter()

# Load the model.
dna_model = dna_client.create(userdata.get('ALPHA_GENOME_API_KEY'))

In [25]:
# Single variant
variant = genome.Variant(
    chromosome='chr13',
    position=94706909,
    reference_bases='G',
    alternate_bases='A'
)

interval = variant.reference_interval.resize(dna_client.SEQUENCE_LENGTH_1MB)

variant_output = dna_model.predict_variant(
    interval=interval,
    variant=variant,
    requested_outputs=[dna_client.OutputType.RNA_SEQ],
    ontology_terms=['UBERON:0000955'],
)  # Brain


variant_output.reference.rna_seq # TrackData
variant_output.reference.rna_seq.metadata
variant_output.reference.rna_seq.values # track predictions


array([[4.6253204e-05, 1.9073486e-06],
       [7.3432922e-05, 2.8759241e-06],
       [1.1205673e-04, 3.9935112e-06],
       ...,
       [5.5506825e-07, 1.8119812e-05],
       [2.1010637e-06, 6.0558319e-05],
       [4.1127205e-06, 1.0871887e-04]], dtype=float32)

In [27]:

vcf_file = """variant_id\tCHROM\tPOS\tREF\tALT
SP0168279_chr13_94706909_G_A_WGS1\tchr13\t94706909\tG\tA
SP0290557_chr13_94707484_C_T_WGS4\tchr13\t94707484\tC\tT
SP0039814_chr13_94706430_A_G_WGS5\tchr13\t94706430\tA\tG
"""
vcf = pd.read_csv(StringIO(vcf_file), sep='\t')

required_columns = ['variant_id', 'CHROM', 'POS', 'REF', 'ALT']
for column in required_columns:
  if column not in vcf.columns:
    raise ValueError(f'VCF file is missing required column: {column}.')

organism = 'human'

sequence_length = '1MB'
sequence_length = dna_client.SUPPORTED_SEQUENCE_LENGTHS[
    f'SEQUENCE_LENGTH_{sequence_length}'
]

results = []

for i, vcf_row in tqdm(vcf.iterrows(), total=len(vcf)):
  variant = genome.Variant(
      chromosome=str(vcf_row.CHROM),
      position=int(vcf_row.POS),
      reference_bases=vcf_row.REF,
      alternate_bases=vcf_row.ALT,
      name=vcf_row.variant_id,
  )
  interval = variant.reference_interval.resize(sequence_length)
  variant_prediction = dna_model.predict_variant(
    interval=interval,
    variant=variant,
    requested_outputs=[dna_client.OutputType.RNA_SEQ],
    ontology_terms=['UBERON:0000955'],
  )
  results.append(variant_prediction.reference.rna_seq.values)
  results.append(variant_prediction.alternate.rna_seq.values)

print(results)


100%|██████████| 3/3 [00:04<00:00,  1.45s/it]

[array([[4.6253204e-05, 1.9073486e-06],
       [7.3432922e-05, 2.8759241e-06],
       [1.1205673e-04, 3.9935112e-06],
       ...,
       [5.5506825e-07, 1.8119812e-05],
       [2.1010637e-06, 6.0558319e-05],
       [4.1127205e-06, 1.0871887e-04]], dtype=float32), array([[4.6253204e-05, 1.9073486e-06],
       [7.6293945e-05, 2.8759241e-06],
       [1.1205673e-04, 3.9935112e-06],
       ...,
       [5.5506825e-07, 1.9550323e-05],
       [2.1010637e-06, 6.0558319e-05],
       [4.1127205e-06, 1.0871887e-04]], dtype=float32), array([[7.3432922e-05, 2.2202730e-06],
       [1.2683868e-04, 3.6954880e-06],
       [1.9168854e-04, 5.1558018e-06],
       ...,
       [2.9504299e-06, 4.1484833e-05],
       [1.0311604e-05, 1.3351440e-04],
       [1.6927719e-05, 2.3078918e-04]], dtype=float32), array([[7.3432922e-05, 2.2202730e-06],
       [1.2683868e-04, 3.6954880e-06],
       [1.9168854e-04, 5.1558018e-06],
       ...,
       [2.9504299e-06, 4.1484833e-05],
       [1.0311604e-05, 1.3351440e-04],
   

In [28]:
score_rna_seq = True  # @param { type: "boolean"}
score_cage = True  # @param { type: "boolean" }
score_procap = True  # @param { type: "boolean" }
score_atac = True  # @param { type: "boolean" }
score_dnase = True  # @param { type: "boolean" }
score_chip_histone = True  # @param { type: "boolean" }
score_chip_tf = True  # @param { type: "boolean" }
score_polyadenylation = True  # @param { type: "boolean" }
score_splice_sites = True  # @param { type: "boolean" }
score_splice_site_usage = True  # @param { type: "boolean" }
score_splice_junctions = True  # @param { type: "boolean" }

download_predictions = False  # @param { type: "boolean" }

organism_map = {
    'human': dna_client.Organism.HOMO_SAPIENS,
    'mouse': dna_client.Organism.MUS_MUSCULUS,
}
organism = organism_map[organism]

scorer_selections = {
    'rna_seq': score_rna_seq,
    'cage': score_cage,
    'procap': score_procap,
    'atac': score_atac,
    'dnase': score_dnase,
    'chip_histone': score_chip_histone,
    'chip_tf': score_chip_tf,
    'polyadenylation': score_polyadenylation,
    'splice_sites': score_splice_sites,
    'splice_site_usage': score_splice_site_usage,
    'splice_junctions': score_splice_junctions,
}

all_scorers = variant_scorers.RECOMMENDED_VARIANT_SCORERS
selected_scorers = [
    all_scorers[key]
    for key in all_scorers
    if scorer_selections.get(key.lower(), False)
]

# Remove any scorers or output types that are not supported for the chosen organism.
unsupported_scorers = [
    scorer
    for scorer in selected_scorers
    if (
        organism.value
        not in variant_scorers.SUPPORTED_ORGANISMS[scorer.base_variant_scorer]
    )
    | (
        (scorer.requested_output == dna_client.OutputType.PROCAP)
        & (organism == dna_client.Organism.MUS_MUSCULUS)
    )
]
if len(unsupported_scorers) > 0:
  print(
      f'Excluding {unsupported_scorers} scorers as they are not supported for'
      f' {organism}.'
  )
  for unsupported_scorer in unsupported_scorers:
    selected_scorers.remove(unsupported_scorer)


# Score variants in the VCF file.
results = []

for i, vcf_row in tqdm(vcf.iterrows(), total=len(vcf)):
  variant = genome.Variant(
      chromosome=str(vcf_row.CHROM),
      position=int(vcf_row.POS),
      reference_bases=vcf_row.REF,
      alternate_bases=vcf_row.ALT,
      name=vcf_row.variant_id,
  )
  interval = variant.reference_interval.resize(sequence_length)

  variant_scores = dna_model.score_variant(
      interval=interval,
      variant=variant,
      variant_scorers=selected_scorers,
      organism=organism,
  )
  results.append(variant_scores)

df_scores = variant_scorers.tidy_scores(results)

if download_predictions:
  df_scores.to_csv('variant_scores.csv', index=False)
  files.download('variant_scores.csv')

df_scores

100%|██████████| 3/3 [00:06<00:00,  2.05s/it]


,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,track_strand,Assay title,ontology_curie,biosample_name,biosample_type,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
0,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000084,T-cell,primary_cell,NaN,NaN,NaN,-0.017078,-0.599367
1,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000100,motor neuron,in_vitro_differentiated_cells,NaN,NaN,NaN,0.013206,0.291254
2,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000236,B cell,primary_cell,NaN,NaN,NaN,-0.014325,-0.511872
3,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000623,natural killer cell,primary_cell,NaN,NaN,NaN,0.000499,-0.046111
4,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,.,ATAC-seq,CL:0000624,"CD4-positive, alpha-beta T cell",primary_cell,NaN,NaN,NaN,-0.025337,-0.721776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31480,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000260962,LINC00557,lncRNA,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,.,polyA plus RNA-seq,UBERON:0036149,suprapubic skin,tissue,NaN,NaN,Skin_Not_Sun_Exposed_Suprapubic,-0.000340,-0.270000
31481,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000274168,RN7SL585P,misc_RNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,.,polyA plus RNA-seq,UBERON:0036149,suprapubic skin,tissue,NaN,NaN,Skin_Not_Sun_Exposed_Suprapubic,-0.008290,-0.992509
31482,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000287635,ENSG00000287635,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,.,polyA plus RNA-seq,UBERON:0036149,suprapubic skin,tissue,NaN,NaN,Skin_Not_Sun_Exposed_Suprapubic,-0.000303,-0.237625
31483,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000289204,ENSG00000289204,lncRNA,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,.,polyA plus RNA-seq,UBERON:0036149,suprapubic skin,tissue,NaN,NaN,Skin_Not_Sun_Exposed_Suprapubic,0.000337,0.270000


In [29]:
columns = [c for c in df_scores.columns if c != 'ontology_curie']
df_scores[(df_scores['ontology_curie'] == 'UBERON:0000955')][columns]

,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,track_name,track_strand,Assay title,biosample_name,biosample_type,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
388,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,DNASE,"CenterMaskScorer(requested_output=DNASE, width...",UBERON:0000955 DNase-seq,.,DNase-seq,brain,tissue,NaN,NaN,NaN,-0.038481,-7.379445e-01
1994,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,CHIP_TF,"CenterMaskScorer(requested_output=CHIP_TF, wid...",UBERON:0000955 TF ChIP-seq CTCF,.,TF ChIP-seq,brain,organoid,CTCF,NaN,NaN,0.025267,8.535151e-01
2887,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,CHIP_HISTONE,CenterMaskScorer(requested_output=CHIP_HISTONE...,UBERON:0000955 Histone ChIP-seq H3K27me3,.,Histone ChIP-seq,brain,tissue,NaN,H3K27me3,NaN,-0.001413,-2.267112e-01
2888,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,CHIP_HISTONE,CenterMaskScorer(requested_output=CHIP_HISTONE...,UBERON:0000955 Histone ChIP-seq H3K36me3,.,Histone ChIP-seq,brain,tissue,NaN,H3K36me3,NaN,0.000324,1.137757e-12
2889,chr13:94706909:G>A,chr13:94182621-95231197:.,None,None,None,None,None,None,CHIP_HISTONE,CenterMaskScorer(requested_output=CHIP_HISTONE...,UBERON:0000955 Histone ChIP-seq H3K4me1,.,Histone ChIP-seq,brain,tissue,NaN,H3K4me1,NaN,-0.001354,-2.047139e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28323,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000237924,RPL21P112,processed_pseudogene,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),UBERON:0000955 polyA plus RNA-seq,-,polyA plus RNA-seq,brain,tissue,NaN,NaN,,-0.000499,-4.218197e-01
28324,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000238230,LINC00391,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),UBERON:0000955 polyA plus RNA-seq,-,polyA plus RNA-seq,brain,tissue,NaN,NaN,,-0.009642,-9.954839e-01
28325,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000274168,RN7SL585P,misc_RNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),UBERON:0000955 polyA plus RNA-seq,-,polyA plus RNA-seq,brain,tissue,NaN,NaN,,-0.012149,-9.975752e-01
28326,chr13:94706430:A>G,chr13:94182142-95230718:.,ENSG00000287635,ENSG00000287635,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),UBERON:0000955 polyA plus RNA-seq,-,polyA plus RNA-seq,brain,tissue,NaN,NaN,,0.000579,4.680602e-01
